# L5: Automate Event Planning

In this lesson, you will learn more about Tasks.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Crew, Task

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key,get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [6]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [7]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

## Creating Venue Pydantic Object

- Create a class `VenueDetails` using [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/).
- Agents will populate this object with information about different venues by creating different instances of it.

In [8]:
from pydantic import BaseModel
# Define a Pydantic model for venue details
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## Creating Tasks
- By using `output_json`, you can specify the structure of the output you want.
- By using `output_file`, you can get your output in a file.
- By setting `human_input=True`, the task will ask for human feedback (whether you like the results or not) before finalising it.

In [9]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

- By setting `async_execution=True`, it means the task can run in parallel with the tasks which come after it.

In [10]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [11]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

## Creating the Crew

**Note**: Since you set `async_execution=True` for `logistics_task` and `marketing_task` tasks, now the order for them does not matter in the `tasks` list.

In [12]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator,
            logistics_manager,
            marketing_communications_agent],

    tasks=[venue_task,
           logistics_task,
           marketing_task],

    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [13]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

**Note 1**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

**Note 2**: 
- Since you set `human_input=True` for some tasks, the execution will ask for your input before it finishes running.
- When it asks for feedback, use your mouse pointer to first click in the text box before typing anything.

In [14]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
I need to find a venue in San Francisco that meets the criteria for the Tech Innovation Conference. I must consider the theme, size, and budget constraints of the event.

Action: Search the internet
Action Input: {"search_query": "venues for events in San Francisco"} 


Search results: Title: Party Venues in San Francisco, CA - Eventective
Link: https://www.eventective.com/san-francisco-ca/party-event-venues/
Snippet: There are 413 Party Venues in and around San Francisco at Eventective.com. Use the filter options to find your Perfect San Francisco Party Venue.
---
Title: Top 20 Venues in San Francisco, CA - PartySlate
Link: https://www.partyslate.com/find-venues/near/san-francisco-ca-usa
Snippet: Top Venues in San Francisco ; San Francisco Design Center Galleria. 101 Henry Adams St, San Fr

This is the agent final answer: Based on the search results and the information from the Bizzabo website, I have identified several potential venues in San Francisco that could accommodate the Tech Innovation Conference. Here are some of the top options:

1. City View at METREON
Location: SoMa
Capacity: 2,000
Website: www.cityviewmetreon.com

2. The Village Venue
Location: Mid-Market
Capacity: 1,100
Website: www.969market.com

3. Exploratorium
Location: Pier 15
Capacity: 3,500
Website: www.exploratorium.edu/rentals

4. The Fillmore
Location: Fillmore District
Capacity: 800
Website: www.livenation.com/venue

5. Gallery 308 at Fort Mason
Location: Marina District
Capacity: 430
Website: https://fortmason.org/venue/gallery-308

6. Bespoke
Location: Downtown
Capacity: 1,200
Website: www.bespokesf.co/events

7. Merchants Exchange’s Julia Morgan Ballroom
Location: Financial District
Capacity: 1,000
Website: https://juliamorganballroom.com

8. The Bently Reserve
Location: Downtown
Capacity: 80

Thought: I have found information about catering and equipment options for the event at City View at METREON.

Action: Read website content
Action Input: {"website_url": "https://www.cityviewmetreon.com/vendors"} 

Vendors — City View at Metreon
 Home
 Venue
 Gallery
 Vendors
 Additional Spaces & Marketing
 Book Your Event
HomeVenueGalleryVendorsAdditional Spaces & Marketing
Book Your Event
Vendors
audio + visual
rentals + decor
catering
tenting
beverage catering
entertainment
floral
photo + video
wedding
additional/misc.
City View at METREON
Subscribe to our mailing list to receive updates and information on upcoming events.
First Name
Last Name
Email Address
Submit
Thank you!
HomeBook Your EventContactDownloadsFAQMetreon

Final Answer: 

City View at METREON in SoMa, San Francisco is a newly renovated event venue with over 31,000 square feet of divisible space that can accommodate up to 2,000 guests. The venue boasts floor-to-ceiling windows with an unobstructed view of San Francisco

Exception in thread Thread-5 (_execute):
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/local/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/site-packages/crewai/task.py", line 182, in _execute
    result = agent.execute_task(
             ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/crewai/agent.py", line 221, in execute_task
    result = self.agent_executor.invoke(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/langchain/chains/base.py", line 163, in invoke
    raise e
  File "/usr/local/lib/python3.11/site-packages/langchain/chains/base.py", line 153, in invoke
    self._call(inputs, run_manager=run_manager)
  File "/us

Thought: I now have the necessary information to effectively market the Tech Innovation Conference and engage at least 500 potential attendees.

Action: Search the internet
Action Input: {
  "search_query": "Tech Innovation Conference in San Francisco"
} 


Search results: Title: Top 12 tech conferences in San Francisco for 2024 - TravelPerk
Link: https://www.travelperk.com/blog/top-tech-conferences-in-san-francisco/
Snippet: With over 200 speakers, the San Francisco Tech Summit offers an action-packed agenda for inspiration, insights, innovation, and connections.
---
Title: Discover Tech Conferences Events & Activities in San Francisco, CA
Link: https://www.eventbrite.com/d/ca--san-francisco/tech-conferences/
Snippet: Tech conferences events in San Francisco, CA · French Tech San Francisco - Pre-Summer Party · San Francisco Tech Networking I Executive Order - 5/28 · Out in Tech ...
---
Title: World Agri-Tech Innovation Summit, March 11-12, 2025
Link: https://worldagritechusa.com/
Snip

Thought: I now can give a great answer
Final Answer: 

# Tech Innovation Conference Marketing Report

## Marketing Activities
- Researched Tech Innovation Conferences in San Francisco to understand the competition landscape.
- Verified the availability of City View at METREON in SoMa, San Francisco as the event venue.
- Explored Eventbrite for tech conference events in San Francisco to identify potential attendees.
- Gathered information on various tech events happening in San Francisco to gauge interest and engagement.
- Reviewed the website content of City View at METREON to understand the venue's capabilities and appeal.

## Attendee Engagement
- Identified multiple tech events taking place in San Francisco related to innovation and technology.
- Found events catering to various tech niches, providing insights into attendee preferences.
- Discovered networking opportunities and tech-related activities in the San Francisco area.
- Explored different event formats and pricing options 

- Display the generated `venue_details.json` file.

In [17]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': 'SoMa, San Francisco',
 'booking_status': 'Available',
 'capacity': 2000,
 'name': 'City View at METREON'}


- Display the generated `marketing_report.md` file.

**Note**: After `kickoff` execution has successfully ran, wait an extra 45 seconds for the `marketing_report.md` file to be generated. If you try to run the code below before the file has been generated, your output would look like:

```
marketing_report.md
```

If you see this output, wait some more and than try again.

In [18]:
from IPython.display import Markdown
Markdown("marketing_report.md")

# Tech Innovation Conference Marketing Report

## Marketing Activities
- Researched Tech Innovation Conferences in San Francisco to understand the competition landscape.
- Verified the availability of City View at METREON in SoMa, San Francisco as the event venue.
- Explored Eventbrite for tech conference events in San Francisco to identify potential attendees.
- Gathered information on various tech events happening in San Francisco to gauge interest and engagement.
- Reviewed the website content of City View at METREON to understand the venue's capabilities and appeal.

## Attendee Engagement
- Identified multiple tech events taking place in San Francisco related to innovation and technology.
- Found events catering to various tech niches, providing insights into attendee preferences.
- Discovered networking opportunities and tech-related activities in the San Francisco area.
- Explored different event formats and pricing options to attract diverse attendees.
- Analyzed trends in San Francisco to align the Tech Innovation Conference with the local tech community's interests.

Overall, the marketing activities have provided valuable insights into the competitive landscape and attendee preferences, enabling us to tailor our promotional strategies effectively and engage with the target audience to achieve our goal of 500 potential attendees for the Tech Innovation Conference.